In [1]:
import sys
import os
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import yaml
from datetime import datetime as dt, date
import requests
import logging
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.databaseService.Merger import Merger
from src.databaseService.Parser import Parser_AV
from src.common.AssetDataService import AssetDataService

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]
# Usage
logger.info("This will print to the notebook's output cell")

This will print to the notebook's output cell


In [2]:
# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [3]:
ticker = "AAPL"
ts = TimeSeries(key=apiKey, output_format='pandas')
fd = FundamentalData(key=apiKey, output_format='pandas')

datashareprice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+ticker+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+ticker+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+apiKey
earningsData = requests.get(url).json()

if incStatementData=={} or cashFlowData == {} or balanceSheetData == {} or earningsData == {}:
    raise ImportError(f"Empty Financial Data")

tickershareprice = Parser_AV(sharepriceData = datashareprice).parse_shareprice()
tickershareprice_dated = tickershareprice.copy()
tickershareprice_dated["Date"] = tickershareprice_dated["Date"].apply(lambda x: dt.strptime(x, '%Y-%m-%d').date())
mask = (tickershareprice_dated["Date"] >= date(2024, 1, 1)) & \
       (tickershareprice_dated["Date"] <= date(2024, 12, 31))
tickershareprice_dated = tickershareprice_dated.loc[mask].reset_index(drop=True)
tickershareprice_dated["Date"] = tickershareprice_dated["Date"].apply(lambda x: str(x))
tickershareprice_dated = tickershareprice_dated.astype({'Date': pd.StringDtype(storage="python")})
tickerfinan, tickerfinquar = Parser_AV(
    incStatementData = incStatementData, 
    cashFlowData = cashFlowData, 
    balanceSheetData = balanceSheetData, 
    earningsData = earningsData
).parse_financials()

## Mock instances

In [4]:
price_cols = ['Date','Open','High','Low','Close','AdjClose','Volume','Dividends','Splits']
finq_cols = ['fiscalDateEnding','reportedDate','reportedEPS','estimatedEPS',
             'surprise','surprisePercentage','reportTime','grossProfit','totalRevenue',
             'ebit','ebitda','totalAssets','totalCurrentLiabilities',
             'totalShareholderEquity','commonStockSharesOutstanding','operatingCashflow']
fina_cols = ['fiscalDateEnding','reportedEPS','grossProfit','totalRevenue',
             'ebit','ebitda','totalAssets','totalCurrentLiabilities',
             'totalShareholderEquity','operatingCashflow']
shareprice_dtype_dict = {'Date': 'string','Open': 'Float64', 'High': 'Float64','Low': 'Float64', 
                         'Close': 'Float64','AdjClose': 'Float64','Volume': 'Float64',
                         'Dividends': 'Float64','Splits': 'Float64'}
finq_dtype_dict = {'fiscalDateEnding': 'string','reportedDate': 'string','reportedEPS': 'Float64','estimatedEPS': 'Float64',
                  'surprise': 'Float64','surprisePercentage': 'Float64','reportTime': 'string','grossProfit': 'Float64','totalRevenue': 'Float64',
                  'ebit': 'Float64','ebitda': 'Float64','totalAssets': 'Float64','totalCurrentLiabilities': 'Float64',
                  'totalShareholderEquity': 'Float64','commonStockSharesOutstanding': 'Float64','operatingCashflow': 'Float64'}
fina_dtype_dict = {'fiscalDateEnding': 'string','reportedEPS': 'Float64','grossProfit': 'Float64','totalRevenue': 'Float64',
                  'ebit': 'Float64','ebitda': 'Float64','totalAssets': 'Float64','totalCurrentLiabilities': 'Float64',
                  'totalShareholderEquity': 'Float64','operatingCashflow': 'Float64'}


# 1. Empty instance
asset_empty = AssetDataService.defaultInstance(ticker="EMPTY")

# 2. Single-row instance
single_price = pd.DataFrame([{
    'Date': '2024-03-28','Open': 100.0,'High': 105.0,'Low': 99.0,'Close': 104.0,'AdjClose': 104.0,'Volume': 1_000_000,'Dividends': 0.5,'Splits': 0.0
}])
single_finq = pd.DataFrame([{
    'fiscalDateEnding': '2024-03-31','reportedDate': '2024-04-15','reportedEPS': 1.2,'estimatedEPS': 1.1,'surprise': 0.1,'surprisePercentage': 9.1,'reportTime': 'post-market','grossProfit': 500_000,'totalRevenue': 1_000_000,'ebit': 200_000,'ebitda': 250_000,'totalAssets': 5_000_000,'totalCurrentLiabilities': 1_000_000,'totalShareholderEquity': 4_000_000,'commonStockSharesOutstanding': 10_000_000,'operatingCashflow': 150_000
}])
single_fina = pd.DataFrame([{
    'fiscalDateEnding': '2024-12-31','reportedEPS': 4.8,'grossProfit': 2_000_000,'totalRevenue': 4_000_000,'ebit': 800_000,'ebitda': 1_000_000,'totalAssets': 20_000_000,'totalCurrentLiabilities': 5_000_000,'totalShareholderEquity': 15_000_000,'operatingCashflow': 600_000
}])
asset_single = AssetData(ticker="ONE", shareprice=single_price,
                         financials_quarterly=single_finq,
                         financials_annually=single_fina)
asset_single.shareprice = asset_single.shareprice.astype(shareprice_dtype_dict)
asset_single.financials_annually = asset_single.financials_annually.astype(fina_dtype_dict)
asset_single.financials_quarterly = asset_single.financials_quarterly.astype(finq_dtype_dict)

# 2. double-row instance
double_price = pd.DataFrame([
    {'Date': '2023-03-28','Open': 100.0,'High': 105.0,'Low': 99.0,'Close': 104.0,'AdjClose': 104.0,'Volume': 1_000_000.0,'Dividends': 0.5,'Splits': 0.0},
    {'Date':'2024-01-10','Open': 5000.0,'High': 55.0,'Low':49.0,'Close':54.0,'AdjClose':54.0,'Volume':500_000.0,'Dividends':0.0,'Splits':0.0},

])
double_finq = pd.DataFrame([
    {'fiscalDateEnding': '2023-03-30','reportedDate': '2024-04-15','reportedEPS': 1.2,'estimatedEPS': 1.1,'surprise': 0.1,'surprisePercentage': 9.1,'reportTime': 'post-market','grossProfit': 500_000,'totalRevenue': 1_000_000,'ebit': 200_000,'ebitda': 250_000,'totalAssets': 5_000_000,'totalCurrentLiabilities': 1_000_000,'totalShareholderEquity': 4_000_000,'commonStockSharesOutstanding': 10_000_000,'operatingCashflow': 150_000},
    {'fiscalDateEnding':'2024-03-31','reportedDate':'2024-04-15','reportedEPS':0.08,'estimatedEPS':0.7,'surprise':0.1,'surprisePercentage':14.3,'reportTime':'pre-market','grossProfit':300_000,'totalRevenue':600_000,'ebit':100_000,'ebitda':120_000,'totalAssets':3_000_000,'totalCurrentLiabilities':800_000,'totalShareholderEquity':2_200_000,'commonStockSharesOutstanding':8_000_000,'operatingCashflow':90_000},
])

double_fina = pd.DataFrame([
    {'fiscalDateEnding':'2024-01-31','reportedEPS': 4.8,'grossProfit': 2_000_000,'totalRevenue': 4_000_000,'ebit': 800_000,'ebitda': 1_000_000,'totalAssets': 20_000_000,'totalCurrentLiabilities': 5_000_000,'totalShareholderEquity': 15_000_000,'operatingCashflow': 600_000},
    {'fiscalDateEnding':'2024-08-31','reportedEPS':30.7,'grossProfit':1_300_000,'totalRevenue':2_600_000,'ebit':520_000,'ebitda':620_000,'totalAssets':12_500_000,'totalCurrentLiabilities':3_100_000,'totalShareholderEquity':9_400_000,'operatingCashflow':420_000},
])
asset_double = AssetData(ticker="ONE", shareprice=double_price,
                         financials_quarterly=double_finq,
                         financials_annually=double_fina)
asset_double.shareprice = asset_double.shareprice.astype(shareprice_dtype_dict)
asset_double.financials_annually = asset_double.financials_annually.astype(fina_dtype_dict)
asset_double.financials_quarterly = asset_double.financials_quarterly.astype(finq_dtype_dict)

# 3. Three-row instance
three_price = pd.DataFrame([
    {'Date':'2024-01-10','Open': 50,'High': 55,'Low':49,'Close':54,'AdjClose':54,'Volume':pd.NA,'Dividends':0,'Splits':0},
    {'Date':'2024-02-10','Open': 52,'High': 57,'Low':51,'Close':56,'AdjClose':56,'Volume':600_000,'Dividends':0,'Splits':0},
    {'Date':'2024-03-10','Open': 54,'High': 59,'Low':53,'Close':58,'AdjClose':58,'Volume':700_000,'Dividends':0,'Splits':0}
])
three_finq = pd.DataFrame([
    {'fiscalDateEnding':'2024-03-31','reportedDate':'2024-04-15','reportedEPS':0.8,'estimatedEPS':pd.NA,'surprise':0.1,'surprisePercentage':14.3,'reportTime':'pre-market','grossProfit':300_000,'totalRevenue':600_000,'ebit':100_000,'ebitda':120_000,'totalAssets':3_000_000,'totalCurrentLiabilities':800_000,'totalShareholderEquity':2_200_000,'commonStockSharesOutstanding':8_000_000,'operatingCashflow':90_000},
    {'fiscalDateEnding':'2024-06-30','reportedDate':'2024-07-15','reportedEPS':0.9,'estimatedEPS':0.85,'surprise':0.05,'surprisePercentage':5.9,'reportTime':'post-market','grossProfit':320_000,'totalRevenue':640_000,'ebit':110_000,'ebitda':130_000,'totalAssets':3_200_000,'totalCurrentLiabilities':820_000,'totalShareholderEquity':2_380_000,'commonStockSharesOutstanding':8_000_000,'operatingCashflow':95_000},
    {'fiscalDateEnding':'2024-09-30','reportedDate':'2024-10-15','reportedEPS':1.0,'estimatedEPS':0.95,'surprise':0.05,'surprisePercentage':5.3,'reportTime':'post-market','grossProfit':340_000,'totalRevenue':680_000,'ebit':120_000,'ebitda':140_000,'totalAssets':3_400_000,'totalCurrentLiabilities':840_000,'totalShareholderEquity':2_560_000,'commonStockSharesOutstanding':8_000_000,'operatingCashflow':100_000}
])
three_fina = pd.DataFrame([
    {'fiscalDateEnding':'2024-04-30','reportedEPS':3.5,'grossProfit':1_200_000,'totalRevenue':2_400_000,'ebit':500_000,'ebitda':600_000,'totalAssets':12_000_000,'totalCurrentLiabilities':3_000_000,'totalShareholderEquity':9_000_000,'operatingCashflow':400_000},
    {'fiscalDateEnding':'2024-08-31','reportedEPS':3.7,'grossProfit':1_300_000,'totalRevenue':pd.NA    ,'ebit':520_000,'ebitda':620_000,'totalAssets':12_500_000,'totalCurrentLiabilities':3_100_000,'totalShareholderEquity':9_400_000,'operatingCashflow':420_000},
    {'fiscalDateEnding':'2024-12-31','reportedEPS':3.9,'grossProfit':1_400_000,'totalRevenue':2_800_000,'ebit':540_000,'ebitda':640_000,'totalAssets':13_000_000,'totalCurrentLiabilities':3_200_000,'totalShareholderEquity':9_800_000,'operatingCashflow':440_000}
])
asset_three = AssetData(ticker="THREE", shareprice=three_price,
                        financials_quarterly=three_finq,
                        financials_annually=three_fina)
asset_three.shareprice = asset_three.shareprice.astype(shareprice_dtype_dict)
asset_three.financials_annually = asset_three.financials_annually.astype(fina_dtype_dict)
asset_three.financials_quarterly = asset_three.financials_quarterly.astype(finq_dtype_dict)

# 4 Combined instance
comba_price = pd.DataFrame([
    {'Date':'2023-03-28','Open': 100.0,'High': 105.0,'Low': 99.0,'Close': 104.0,'AdjClose': pd.NA,'Volume': 1_000_000,'Dividends': 0.5,'Splits': 0.0},
    {'Date':'2024-01-10','Open': 50,'High': 55,'Low':49,         'Close':54,'AdjClose':54,'Volume':500_000,'Dividends':0,'Splits':0},
    {'Date':'2024-02-10','Open': 52,'High': 57,'Low':51,         'Close':56,'AdjClose':56,'Volume':600_000,'Dividends':0,'Splits':0},
    {'Date':'2024-03-10','Open': 54,'High': 59,'Low':53,         'Close':58,'AdjClose':58,'Volume':700_000,'Dividends':0,'Splits':0}
])
comba_finq = pd.DataFrame([
    {'fiscalDateEnding':'2023-03-30','reportedDate': '2024-04-15','reportedEPS': 1.2,'estimatedEPS': 1.1,'surprise': 0.1,'surprisePercentage': 9.1,'reportTime': 'post-market','grossProfit': 500_000,'totalRevenue': 1_000_000,'ebit': 200_000,'ebitda': 250_000,'totalAssets': 5_000_000,'totalCurrentLiabilities': 1_000_000,'totalShareholderEquity': 4_000_000,'commonStockSharesOutstanding': 10_000_000,'operatingCashflow': 150_000},
    {'fiscalDateEnding':'2024-03-31','reportedDate':'2024-04-15','reportedEPS':0.8,'estimatedEPS':0.7,'surprise':0.1,'surprisePercentage':14.3,'reportTime':'pre-market','grossProfit':300_000,'totalRevenue':600_000,'ebit':100_000,'ebitda':120_000,'totalAssets':3_000_000,'totalCurrentLiabilities':800_000,'totalShareholderEquity':2_200_000,'commonStockSharesOutstanding':8_000_000,'operatingCashflow':90_000},
    {'fiscalDateEnding':'2024-06-30','reportedDate':'2024-07-15','reportedEPS':0.9,'estimatedEPS':0.85,'surprise':0.05,'surprisePercentage':5.9,'reportTime':'post-market','grossProfit':320_000,'totalRevenue':640_000,'ebit':110_000,'ebitda':130_000,'totalAssets':3_200_000,'totalCurrentLiabilities':820_000,'totalShareholderEquity':2_380_000,'commonStockSharesOutstanding':8_000_000,'operatingCashflow':95_000},
    {'fiscalDateEnding':'2024-09-30','reportedDate':'2024-10-15','reportedEPS':1.0,'estimatedEPS':0.95,'surprise':0.05,'surprisePercentage':5.3,'reportTime':'post-market','grossProfit':340_000,'totalRevenue':680_000,'ebit':120_000,'ebitda':140_000,'totalAssets':3_400_000,'totalCurrentLiabilities':840_000,'totalShareholderEquity':2_560_000,'commonStockSharesOutstanding':8_000_000,'operatingCashflow':100_000}
])
comba_fina = pd.DataFrame([
    {'fiscalDateEnding':'2024-01-31','reportedEPS': 4.8,'grossProfit': 2_000_000,'totalRevenue': 4_000_000,'ebit': 800_000,'ebitda': 1_000_000,'totalAssets': 20_000_000,'totalCurrentLiabilities': 5_000_000,'totalShareholderEquity': 15_000_000,'operatingCashflow': 600_000},
    {'fiscalDateEnding':'2024-04-30','reportedEPS':3.5,'grossProfit':1_200_000,'totalRevenue':2_400_000,'ebit':500_000,'ebitda':600_000,'totalAssets':12_000_000,'totalCurrentLiabilities':3_000_000,'totalShareholderEquity':9_000_000,'operatingCashflow':400_000},
    {'fiscalDateEnding':'2024-08-31','reportedEPS':3.7,'grossProfit':1_300_000,'totalRevenue':2_600_000,'ebit':520_000,'ebitda':620_000,'totalAssets':12_500_000,'totalCurrentLiabilities':3_100_000,'totalShareholderEquity':9_400_000,'operatingCashflow':420_000},
    {'fiscalDateEnding':'2024-12-31','reportedEPS':3.9,'grossProfit':1_400_000,'totalRevenue':2_800_000,'ebit':540_000,'ebitda':640_000,'totalAssets':13_000_000,'totalCurrentLiabilities':3_200_000,'totalShareholderEquity':9_800_000,'operatingCashflow':440_000}
])
asset_comba = AssetData(ticker="THREE", shareprice=comba_price,
                        financials_quarterly=comba_finq,
                        financials_annually=comba_fina)
asset_comba.shareprice = asset_comba.shareprice.astype(shareprice_dtype_dict)
asset_comba.financials_annually = asset_comba.financials_annually.astype(fina_dtype_dict)
asset_comba.financials_quarterly = asset_comba.financials_quarterly.astype(finq_dtype_dict)


## Testing merge_shareprice

## Shareprice

In [5]:
# Test 1: init
asset_empty_new = AssetDataService.copy(asset_empty)
merger = Merger(asset_empty_new)
merger.merge_shareprice(tickershareprice_dated)
diff = asset_empty_new.shareprice.compare(tickershareprice_dated)
assert diff.empty, f"Expected no changes, but asset state mutated: {diff}"
assert asset_empty_new.shareprice.columns.equals(tickershareprice_dated.columns), f"Expected no changes in columns, but asset columns mutated {asset_empty_new.shareprice.columns}"
assert asset_empty_new.shareprice.dtypes.equals(tickershareprice_dated.dtypes), f"Expected no changes in dtypes, but asset dtypes mutated {asset_empty_new.shareprice.dtypes}"
print('✅ Test 1 passed: New rows appended correctly.')

# Test 2: single overtake
asset_single_new = AssetDataService.copy(asset_single)
merger = Merger(asset_single_new)
merger.merge_shareprice(tickershareprice_dated)
diff2 = asset_single_new.shareprice.compare(tickershareprice_dated)
assert diff2.empty, f"Expected no changes, but asset state mutated: {diff2}"
assert asset_single_new.shareprice.columns.equals(tickershareprice_dated.columns), f"Expected no changes in columns, but asset columns mutated {asset_single_new.shareprice.columns}"
assert asset_single_new.shareprice.dtypes.equals(tickershareprice_dated.dtypes), f"Expected no changes in dtypes, but asset dtypes mutated {asset_single_new.shareprice.dtypes}"
print('✅ Test 2 passed: Changes single completely.')

# Test 3: change in entry; single
asset_single_new = AssetDataService.copy(asset_single)
asset_single_change = AssetDataService.copy(asset_single_new)
asset_single_change.shareprice.at[0,'Open'] = 10.0
asset_single_change.shareprice = asset_single_change.shareprice.astype(shareprice_dtype_dict)  # Needed due to lazy evaluation in pandas
merger = Merger(asset_single_new)
merger.merge_shareprice(asset_single_change.shareprice)
diff3 = asset_single_new.shareprice.compare(asset_single_change.shareprice)
assert diff3.empty, f"Expected no changes, but asset state mutated: {diff3}"
assert asset_single_new.shareprice.columns.equals(asset_single_change.shareprice.columns), f"Expected no changes in columns, but asset state mutated {asset_single_new.shareprice.columns}"
assert asset_single_new.shareprice.dtypes.equals(asset_single_change.shareprice.dtypes), f"Expected no changes in dtypes, but asset dtypes mutated {asset_single_new.shareprice.dtypes}"
print('✅ Test 3 passed: Changes single completely.')

# Test 4 Comba correct
asset_double_new = AssetDataService.copy(asset_double)
asset_triple_new = AssetDataService.copy(asset_three)
merger = Merger(asset_double_new)
merger.merge_shareprice(asset_triple_new.shareprice)
diff4 = asset_double_new.shareprice.compare(asset_comba.shareprice)
assert diff4.empty, f"Expected no changes, but asset state mutated: {diff4}"
assert asset_double_new.shareprice.columns.equals(asset_comba.shareprice.columns), f"Expected no changes in columns, but asset state mutated {asset_single_new.shareprice.columns}"
assert asset_double_new.shareprice.dtypes.equals(asset_comba.shareprice.dtypes), f"Expected no changes in dtypes, but asset dtypes mutated {asset_single_new.shareprice.dtypes}"
print('✅ Test 4 passed: Combined dataframe correct.')


  No existing shareprice data for ticker EMPTY.
✅ Test 1 passed: New rows appended correctly.
  Open_ratio: 1 values outside +-1%
  High_ratio: 1 values outside +-1%
  Low_ratio: 1 values outside +-1%
  Close_ratio: 1 values outside +-1%
  Splits_ratio: 1 values outside +-1%
  Volume_diff: 1 values outside +-1%
  Dividends_diff: 1 values outside +-1%
  Added 251 new rows to shareprice data of ticker ONE.
✅ Test 2 passed: Changes single completely.
  Open_ratio: 1 values outside +-1%
  Splits_ratio: 1 values outside +-1%
  Added 0 new rows to shareprice data of ticker ONE.
✅ Test 3 passed: Changes single completely.
  Open_ratio: 1 values outside +-1%
  Splits_ratio: 3 values outside +-1%
  Added 2 new rows to shareprice data of ticker ONE.
✅ Test 4 passed: Combined dataframe correct.


## Financials

In [6]:
# Test 1: init
asset_empty_new = AssetDataService.copy(asset_empty)
merger = Merger(asset_empty_new)
merger.merge_financials(fin_quar = tickerfinquar, fin_ann = tickerfinan)
diff_an = asset_empty_new.financials_annually.compare(tickerfinan)
diff_quar = asset_empty_new.financials_quarterly.compare(tickerfinquar)
assert diff_an.empty, f"Expected no changes in annual, but annual financials differ: {diff_an}"
assert diff_quar.empty, f"Expected no changes in quarterly, but quarterly financials differ: {diff_quar}"
assert asset_empty_new.financials_annually.columns.equals(tickerfinan.columns), f"ERROR in columns: {asset_empty_new.financials_annually.columns}"
assert asset_empty_new.financials_quarterly.columns.equals(tickerfinquar.columns), f"ERROR in columns: {asset_empty_new.financials_quarterly.columns}"
assert asset_empty_new.financials_annually.dtypes.equals(tickerfinan.dtypes), f"ERROR in dtypes: {asset_empty_new.financials_annually.dtypes}"
assert asset_empty_new.financials_quarterly.dtypes.equals(tickerfinquar.dtypes), f"ERROR in dtypes: {asset_empty_new.financials_quarterly.dtypes}"
print('✅ Test 1 passed: New rows appended correctly.')

# Test 2: comba correct
asset_double_new = AssetDataService.copy(asset_double)
asset_triple_new = AssetDataService.copy(asset_three)
merger = Merger(asset_double_new)
merger.merge_financials(fin_quar = asset_triple_new.financials_quarterly, fin_ann = asset_triple_new.financials_annually)
diff2_an = asset_double_new.financials_annually.compare(asset_comba.financials_annually)
diff2_quar = asset_double_new.financials_quarterly.compare(asset_comba.financials_quarterly)
assert diff2_an.empty, f"Expected no changes in annual, but annual financials differ: {diff2_an}"
assert diff2_quar.empty, f"Expected no changes in quarterly, but quarterly financials differ: {diff2_quar}"
assert asset_double_new.financials_annually.columns.equals(asset_comba.financials_annually.columns), f"Expected no changes in columns, but asset state mutated {asset_double_new.financials_annually.columns}"
assert asset_double_new.financials_quarterly.columns.equals(asset_comba.financials_quarterly.columns), f"Expected no changes in columns, but asset state mutated {asset_double_new.financials_quarterly.columns}"
assert asset_double_new.financials_annually.dtypes.equals(asset_comba.financials_annually.dtypes), f"Expected no changes in dtypes, but asset dtypes mutated {asset_double_new.financials_annually.dtypes}"
assert asset_double_new.financials_quarterly.dtypes.equals(asset_comba.financials_quarterly.dtypes), f"Expected no changes in dtypes, but asset dtypes mutated {asset_double_new.financials_quarterly.dtypes}"
print('✅ Test 2 passed: Combination correct.')

annual: first load (29 rows)
quarterly: first load (117 rows)
✅ Test 1 passed: New rows appended correctly.
annual: +2 new rows [datetime.date(2024, 4, 30), datetime.date(2024, 12, 31)]
annual diffs: (1 rows, 2 cells)
quarterly: +2 new rows [datetime.date(2024, 6, 30), datetime.date(2024, 9, 30)]
quarterly diffs: (1 rows, 2 cells)
✅ Test 2 passed: Combination correct.
